In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [3]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)

model.pool.register_forward_hook(layer1_hook)


In [4]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [7]:
outputs

[tensor([[[[7.5440e-01, 1.1918e+00, 4.7843e-01,  ..., 6.5146e-01,
            7.3261e-01, 4.4735e-01],
           [0.0000e+00, 9.4595e-01, 9.2253e-01,  ..., 7.0088e-01,
            4.9980e-01, 8.2953e-01],
           [3.2667e-01, 7.5914e-01, 4.2505e-01,  ..., 4.0203e-01,
            1.7237e-01, 1.5553e-01],
           ...,
           [1.4629e+00, 1.2174e+00, 1.4100e-01,  ..., 1.2033e-01,
            1.2340e-01, 5.6060e-01],
           [4.5111e-01, 8.8415e-01, 5.1760e-01,  ..., 8.2607e-01,
            3.5850e-01, 1.2143e+00],
           [5.3102e-01, 5.0105e-01, 4.6788e-01,  ..., 1.0050e+00,
            5.1596e-01, 1.1288e+00]],
 
          [[1.4838e-01, 1.1844e+00, 1.1252e+00,  ..., 1.0391e+00,
            8.1670e-01, 0.0000e+00],
           [9.7518e-01, 5.4812e-01, 9.4987e-01,  ..., 1.0590e+00,
            4.2367e-01, 2.0461e-01],
           [5.2989e-01, 1.0560e+00, 9.0394e-01,  ..., 4.3364e-01,
            6.1023e-01, 6.5913e-01],
           ...,
           [7.9428e-01, 8.5483e-01, 8.

In [10]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [11]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            #"自行填入"
            m.bias.data.fill_(1)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [12]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


